Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Philip Boakye"
COLLABORATORS = ""

---

<h1>CS152 Assignment 3: The DPLL Algorithm</h1>

<h1>Question 1</h1>

Define your <code>Literal</code> class below.  Ensure that you define a <code>name</code> and <code>sign</code> attribute as required in the assignment description.  In addition, include any other attributes and member functions as needed. You will need to overload <code>__neg__()</code> to correctly handle negated literals.  You may need to overload <code>__eq__()</code> and <code>__hash__()</code> also.

In [2]:
# Import any packages you need here
# Also define any variables as needed

# YOUR CODE HERE (OPTIONAL)

class Literal:
    # YOUR CODE HERE
    def __init__(self, name): # assignment demanded we initialized this class with name and sign attribute.
        self.name = name
        self.sign = True #we would need to later consider situations with negative literals
        
    def __neg__(self): # this will handle negative literals
        literal = Literal(self.name)
        literal.sign = not(self.sign)
        return literal
    def __eq__(self, other): # to check equality of two literals
        return self.name == other.name
    def __lt__(self, other): #to check if one literal is less than another
        return self.name < other.name
    def __hash__(self):
        return self.name.__hash__()
    def __repr__(self): # to represent output literals
        if self.sign:sign = ""
        else:sign = "-"
        return f"{sign}{self.name}"
    def pureversion(self): #sometimes we will need the pure version of a literal with no negation
        if not self.sign:return -self
        else:return self       
        
        
        
        

<h1>Question 2</h1>
Implement DPLL by filling in the stubs below.  Note that the various heuristics are not required to be implemented for basic functionality, but a template has been provided for you to do so if you attempt the extension questions

In [3]:
# Define degree, pure symbol and unit clause heuristics here (optional).
# Add in your functions under the templates below
# YOUR CODE HERE (OPTIONAL)


# WE WOULD NEED TO DEFINE EVERY SINGLE FUNCTION THAT WILL BE NEEDED IN DPLL_SATISFIABLE

# FUNCTION 1: WE MIGHT NEED TO SIMPLIFY A CLAUSE BASED ON PROVIDED MODEL
def simplifyClause(clause, model):
    simplifiedClause = set()
    for literal in clause:
        temp = model.get(literal, None)
        if temp and literal.sign or temp==False and literal.sign==False:return None
        elif temp is None:simplifiedClause.add(literal)
    if len(simplifiedClause) == 0:return clause #meaning we were unable to simplify
    return simplifiedClause

# FUNCTION 2: KBs ARE MADE UP OF CLAUSES AND MIGHT NEED TO SIMPLIFY OUR KB BASED ON PROVIDED MODEL
def simplifyKB(clauses, model):
    simplifiedKB = []
    for clause in clauses:
        simplifiedClause = simplifyClause(clause, model)
        if simplifiedClause is not None:simplifiedKB.append(simplifiedClause)
    return simplifiedKB

# FUNCTION 3: EVALUATES IF ALL CLAUSES ARE TRUE OR SOME ARE FALSE OR IF CANNOT BE EVALUATED BASED ON MODEL
def evaluateClauses(clauses, model):
    current_eval = simplifyKB(clauses, model)
    if len(current_eval) == 0:
        return True # when every clause in model is True we would have an empty simplified KB
    else: # now we check instances when everything is false or cannot be evaluated
        for clause in current_eval:
            onlyfalse = True
            for literal in clause:
                if model.get(literal, None) is None:
                    onlyfalse = False
                    break
            if onlyfalse == True:return False
        return None

# FUNCTION 4: GET ALL DISTINCT LITERALS/SYMBOLS IN CLAUSES
def distinctSymbols(clauses):
    distinctsymbols = set() # used symbols because literals are used in mutiple places in this code.
    for clause in clauses:
        for symbol in clause:
            distinctsymbols.add(symbol)
    return distinctsymbols

# FUNCTION 5: GET MOST FREQUENT SYMBOL IN A GIVEN MODEL. WILL BE USED LATER IN HEURISTICS
def mostFrequentSymbol(symbols, clauses, model):
    simplifiedKB = simplifyKB(clauses, model)
    def countSym(symbol):
        total = 0
        for clause in simplifiedKB:
            if symbol in clause:total+=1
        return total
    
    frequencies,frequent={},None
    for symbol in symbols:
        total = countSym(symbol)
        frequencies[symbol] = total
        if frequent is None:frequent=symbol
        else:
            if total>frequencies[frequent]:frequent=symbol
            elif total==frequencies[frequent]:
                if symbol<frequent:frequent=symbol
                else:frequent=frequent
    copiedsymbols=symbols.copy()
    copiedsymbols.remove(frequent)
    return frequent,copiedsymbols

# FUNCTION 6: EXTENSION FUNCTION TO FIND A PURE LITERAL IN A GIVEN KB
def findPureLiteral(symbols, clauses, model, frequent=False):
    simplifiedKB = simplifyKB(clauses, model)
    pureliterals,puresigns,literalsarray = set(),{},list(symbols)
    for i in range(len(symbols)):
        literal=literalsarray[i]
        for clause in simplifiedKB:
            if literal in clause:
                clausearray=list(clause)
                l=clausearray[clausearray.index(literal)]
                if literal in puresigns:
                    if l.sign!=puresigns[literal]:
                        pureliterals.remove(literal)
                        i+=1
                        break
                else:
                    puresigns[literal]=l.sign
                    pureliterals.add(literal)
    if len(pureliterals)==0:pureliteral = None
    elif frequent:pureliteral=mostFrequentSymbol(pureliterals,clauses,model)[0]
    else:pureliteral=sorted(list(pureliterals))[0]
    if pureliteral is None:return None,None
    return pureliteral,puresigns[pureliteral]

# FUNCTION 7: EXTENSION FUNCTION TO FIND UNIT CLAUSE IN GIVEN KB
def findUnitClause(clauses, model, frequent=False):
    simplifiedKB = simplifyKB(clauses, model)
    unitClauses = set()
    for clause in simplifiedKB:
        if len(clause) == 1: #meaning unit clause
            unitClauses.add(clause.pop())
    if len(unitClauses) == 0:unitclause=None
    elif frequent:unitclause=mostFrequentSymbol(unitClauses,clauses,model)[0]
    else:unitclause=sorted(list(unitClauses))[0]
    if unitclause is None:return None,None
    return unitclause,unitclause.sign
    
# WITH THESE FUNCTIONS ABOVE WE SHOULD BE READY TO TACKLE DPLL_SATISFIABLE


def DPLL_Satisfiable(KB, heuristic_level=0):
    ''' satisfiable, model = DPLLSatisfiable(KB)
        Takes in a KB and returns whether the KB is satisfiable, and the model that makes it so
        
        KB: A knowledge base of clauses (CNF) consisting of a list of sets of literals.  A KB might look like
            [{A,B},{-A,C,D}]
        heuristic_level: An integer that will be passed in to specify which heuristics to implement 
            (only for the extension activities).
        satisfiable: Returns True if the KB is satisfiable, or False otherwise
        Model: A dictionary that assigns a truth value to each literal for the model that satisfies KB.
            For example, a model might look like {A: True, B: False}
    '''
    # YOUR CODE HERE
    # FINAL MAIN METHOD THAT DOES THE HEAVY WORK. CHECKS WHETHER A GIVEN KB IS SATISFIABLE AND WHICH MODEL HELPS IT ATTAIN SATISFIABILITY
    def helperDPLL(KB, literals, model, heuristic_level=0, literalarray=[]):
        '''KB: A knowledge base of clauses (CNF) consisting of a list of sets of literals.  A KB might look like
                [{A,B},{-A,C,D}]
        literals: literals in our KB
        model: model used for search
        literalarray: currently selected literals which should be empty from start but later filled with chose symbols for True/False assignment
        heuristic_level: An integer that will be passed in to specify which heuristics to implement 
                (only for the extension activities).
        satisfiable: Returns True if the KB is satisfiable, or False otherwise
        Model: A dictionary that assigns a truth value to each literal for the model that satisfies KB.
                For example, a model might look like {A: True, B: False}
        '''
        #first we evaluate all clauses which also serves as base case for our recursion later on
        current_eval = evaluateClauses(KB, model)
        if current_eval is not None:
            return current_eval, model, literalarray

        # HEURISTICS
        # Degree Hueristic
        if heuristic_level==1:
            literalsarray=list(literals)
            frequent, copiedsymbols = mostFrequentSymbol(literals, KB, model)
            literalarray.append(frequent.pureversion())
            truth=helperDPLL(KB, copiedsymbols, {**model,**{frequent.pureversion():True}},heuristic_level) #** = kwargs and helps to pass in any number of arguments
            if truth[0]:return truth
            nottruth=helperDPLL(KB, copiedsymbols, {**model,**{frequent.pureversion():False}},heuristic_level)
            return nottruth

        # Pure Literal and Unit Clause Heuristic
        elif heuristic_level==2:
            pureliteral, sign = findPureLiteral(literals, KB, model)
            if pureliteral is not None:
                literalarray.append(pureliteral.pureversion())
                copiedliterals=literals.copy()
                copiedliterals.remove(pureliteral)
                return helperDPLL(KB, copiedliterals, {**model,**{pureliteral.pureversion():sign}},heuristic_level)
            unitclause,sign = findUnitClause(KB, model)
            if unitclause is not None:
                literalarray.append(unitclause.pureversion())
                copiedliterals=literals.copy()
                copiedliterals.remove(unitclause)
                return helperDPLL(KB, copiedliterals, {**model,**{unitclause.pureversion():sign}},heuristic_level)

            literalsarray=list(literals)
            frequent, copiedsymbols = mostFrequentSymbol(literals, KB, model)
            literalarray.append(frequent.pureversion())
            truth=helperDPLL(KB, copiedsymbols, {**model,**{frequent.pureversion():True}},heuristic_level)
            if truth[0]:return truth
            nottruth=helperDPLL(KB, copiedsymbols, {**model,**{frequent.pureversion():False}},heuristic_level)
            return nottruth

        elif heuristic_level==3:
            pureliteral, sign = findPureLiteral(literals, KB, model, True)
            if pureliteral is not None:
                literalarray.append(pureliteral.pureversion())
                copiedliterals=literals.copy()
                copiedliterals.remove(pureliteral)
                return helperDPLL(KB, copiedliterals, {**model,**{pureliteral.pureversion():sign}},heuristic_level)
            unitclause,sign = findUnitClause(KB, model, True)
            if unitclause is not None:
                literalarray.append(unitclause.pureversion())
                copiedliterals=literals.copy()
                copiedliterals.remove(unitclause)
                return helperDPLL(KB, copiedliterals, {**model,**{unitclause.pureversion():sign}},heuristic_level)

            literalsarray=list(literals)
            frequent, copiedsymbols = mostFrequentSymbol(literals, KB, model)
            literalarray.append(frequent.pureversion())
            truth=helperDPLL(KB, copiedsymbols, {**model,**{frequent.pureversion():True}},heuristic_level)
            if truth[0]:return truth
            nottruth=helperDPLL(KB, copiedsymbols, {**model,**{frequent.pureversion():False}},heuristic_level)
            return nottruth

        # if heuristic level is not any of the above then per instructions we can assume it to be 0
        else:
            literalsarray=list(literals)
            frequent, copiedsymbols = literalsarray[0],set(literalsarray[1:])
            literalarray.append(frequent.pureversion())
            truth=helperDPLL(KB, copiedsymbols, {**model,**{frequent.pureversion():True}},heuristic_level)
            if truth[0]:return truth
            nottruth=helperDPLL(KB, copiedsymbols, {**model,**{frequent.pureversion():False}},heuristic_level)
            return nottruth 
    
    literals = distinctSymbols(KB)
    return helperDPLL(KB,literals,{}, heuristic_level)
  
    
    

<h1>Question 3</h1>

Implement your KB from Russell & Norvig in CNF by filling in the sets inside the list <code>KB</code> below.  Ensure that your KB is in a list of set format as stated in the assignment instructions.

In [4]:
# Define some literals
A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')

# YOUR CODE HERE
# In Russel and Norvig we had: S1: A <=> (B v E), S2: E => D, S3: C ^ F => ~B, S4: E => B, S5: B => F, S6: B => C
KB = [{-A, B, E}, {A, -B}, {A, -E},#S1
{-E, D},#S2
{-C, -F, -B},#S3
{-E, B},#S4
{-B, F},#S5
{-B, C}#S6    
]
      

<h1>Extensions</h1>

1. Implement the degree heuristic for choosing symbols.  If <code>heuristic_level=1</code>, then the degree heuristic should be used to select which symbol to assign.
2. Implement the pure symbol and unit clause heuristics.  If <code>heuristic_level=2</code>, then these heuristics should be used to find select first a pure symbol, and if one is not found, then a unit clause.  If neither pure symbols nor unit clauses are found, then the degree heuristic should be used to select a symbol.  If there are multiple pure symbols or unit clauses found, then use alphabetical order to select amongst them.
3. Modify the pure symbol heuristic to choose the pure symbol that occurs in the most number of clauses.  This should be activated if <code>heuristic_level=3</code>.

<h1>Basic Functionality Tests</h1>

All of the tests in this section must be passed for the code to be considered fully functional.  Other unseen test cases will be used after submission to further verify functionality

In [5]:
# This section will test the correct definition of the literal class


# Test the name attribute works correctly
assert(A.name == 'A')

# Test that negation works correctly
assert(not (-C).sign)

In [6]:
# This section will test that the KB has been correctly converted to CNF by performing 
# tests over all possible models
import itertools
symbols = {A,B,C,D,E,F}
symbol_list = [A,B,C,D,E,F]

def evaluate_russell_norvig_KB(model):
    # Manually evaluate the KB using the model
    s = list()
    s.append(model[A] == (model[B] or model[E]))
    s.append(model[E] <= model[D])
    s.append((model[C] and model[F]) <= (not model[B]))
    s.append(model[E] <= model[B])
    s.append(model[B] <= model[F])
    s.append(model[B] <= model[C])
    return all(s)

def evaluate_KB(KB, model):
    model_true = True
    model_name_dict = {l.name: model[l] for l in model}
    for clause in KB:
        evaluation = {l.sign == model_name_dict[l.name] for l in clause if l.name in model_name_dict}
        model_true = model_true and any(evaluation)
    return model_true

all_models = list(itertools.product([False, True], repeat=6))
for i in range(0, len(all_models)):
    # Test all truth values
    test_model = {symbol_list[j]: all_models[i][j] for j in range(0,6)}
    
    # Test the model
    assert(evaluate_russell_norvig_KB(test_model) == evaluate_KB(KB, test_model))

In [7]:
# This section will test the basic working of DPLL
# Satisfiable model
test_KB = [{A, C},{-A, C}, {B, -C}]
t, model, symbol_trace = DPLL_Satisfiable(test_KB)
assert(evaluate_KB(test_KB, model))
assert(t)

# Unsatisfiable model
test_KB = [{A, C},{-A, C}, {-C}]
t, model, symbol_trace = DPLL_Satisfiable(test_KB)
assert(not t)

In [8]:
# This will test DPLL on the KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,0)
print(model)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model)) 

{A: False, C: True, F: True, B: False, D: True, E: False}


<h1>Extension Tests</h1>

This section will test that the degree heuristic, pure symbol and unit clause heuristics are correctly implemented.  Note that in your code, the <code>heuristic_level</code> needs to correctly activate the heuristic being tested, and the <code>symbol_trace</code> needs to be correctly generated 

In [9]:
# Degree Heuristic Test for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,1)
print(model)
print(symbol_trace)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))  

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] in [['B','F', 'E', 'C', 'C', 'E', 'A'], 
                                          ['B', 'A', 'C', 'F', 'E', 'A'], 
                                          ['B','E','A']])

symbol_trace

{B: False, E: False, A: False}
[B, A, C, F, E, A]


[B, A, C, F, E, A]

In [10]:
# Pure Symbol & Unit Clause Heuristic Test for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,2)
print(model)
print(symbol_trace)


# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] in [['D', 'B', 'E', 'A', 'C', 'F', 'E', 'A'], 
                                          ['D', 'B', 'A', 'C', 'F', 'E', 'A'], 
                                          ['D', 'B' ,'E', 'A']])

{D: True, B: False, E: False, A: False}
[D, B, A, C, F, E, A]


In [11]:
# Pure Symbol & Unit Clause Heuristic Test, choosing the most-frequently used pure symbol, for KB from Russell & Norvig
t, model, symbol_trace = DPLL_Satisfiable(KB,3)
print(model)
print(symbol_trace)

# This model is satisfiable.  Test that it is so
assert(t)

# Check that the model found actually works
assert(evaluate_KB(KB, model))

# Test the symbol trace to ensure that the correct order is chosen
assert([l.name for l in symbol_trace] in [['D', 'B', 'A', 'C', 'F', 'E', 'A'], 
                                          ['D', 'B' ,'E', 'A']])

{D: True, B: False, E: False, A: False}
[D, B, A, C, F, E, A]


# HCs Applied

**#heuristics:**
Given the basic assignment requirement of implementing a working DPLL, I went ahead to attempt 2/3 other heuristics that can help identify a satisfiable KB and which model helps it.

**#selfawareness:**

For this assignment, I believe I have exhibited a responsible level of #selfawareness by
starting very early ahead of time knowing my weakness of procrastination so I can
finish major portions of the assignment before the break. Also, I noted while it gets to the end of the semester, there can be a burnout and need for extensions, I ensured I kept all my extensions till now so I can actually take time to do this assignment very well. Consequently, this allowed me to consult the TA and my course mates about some obstacles and common confusions. By this, I have been able to commit to my academics and balance my personal plans with this course’s requirements.

**#organization:**

I effectively utilized this HC throughout the assignment. I communicated in a precise
style using terms and diction appropriate to the concepts discussed. I used active voice over passive and avoided pretentious diction. I maintained a descriptive writing style for my explanation, which shows precise detail of analysis. Also for the main DPLL algorithm, before jumping straight into code, I ensured I wrote all the helper functions/methods I would need for that code.